# TSA Complaints Data Quality Check
RR 9/23/23<br>
The goal is to confirm that subcategory totals reconcile with category totals.<br>

analysis notes and assumptions<br>
<ul>
<li>I'm looking at only the master file, master-month.csv
<li><em>Subcategory totals</em> are calculated for each possible combination of category and subcategory, but do not yet consolidate variants of clipped subcategory descriptions.
    <li><em>Category totals</em> for each airport/month are assumed to be rows with null subcategories
</ul>

## 1. import and review data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../output/master-month.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 724710 entries, 0 to 724709
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   PDF Report Date  724710 non-null  object
 1   Airport          708131 non-null  object
 2   Category         686163 non-null  object
 3   Subcategory      463751 non-null  object
 4   Date             724710 non-null  object
 5   Count            724710 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 33.2+ MB


## 2. split dataframe into three dataframes: 1) month counts, 2) category totals, and 3) overall totals

In [4]:
len(df)

724710

In [5]:
# filter to include all rows with a subcategory
df_month_counts = df[df['Subcategory'].notna()]
len(df_month_counts)

463751

In [6]:
# filter to include category totals, i.e. subcategory is blank
df_cat_totals = df[df['Category'].notna() & df['Subcategory'].isna()]
len(df_cat_totals)

222412

In [7]:
# filter to include overall totals, i.e. category and subcategory are blank
df_totals = df[df['Category'].isna() & df['Subcategory'].isna()]
len(df_totals)

38547

In [8]:
#confirm no records are lost or duplicated
len(df_month_counts) + len(df_cat_totals) + len(df_totals)

724710

## 3a. sum incidents by subcategory

In [9]:
#sum by subcategory. this will roll up all time periods and airports
#drop rows with null subcategories, which are really totals
df_summary_subcat = df_month_counts.groupby(['Category','Subcategory'])['Count'].sum().reset_index()
df_summary_subcat.head()

,Category,Subcategory,Count
0,Additional Information Required/Insufficient Inf,Additional Information Required/Insufficien,504
1,Additional Information Required/Insufficient I...,EMAIL ONLY,50
2,Additional Information Required/Insufficient I...,Additional Information Required/Insufficient I...,29
3,Additional Information Required/Insufficient I...,Additional Information Required/Insufficient I...,10
4,Additional Information Required/Insufficient I...,Additional Information Required/Insufficient I...,463


In [10]:
len(df_summary_subcat)

468

## 3b. sum incidents by category

In [12]:
#aggregate by category
df_summary_cat = df_cat_totals.groupby(['Category'])['Count'].sum().reset_index()
df_summary_cat.head()

,Category,Count
0,Additional Information Required/Insufficient Inf,504
1,Additional Information Required/Insufficient I...,50
2,Additional Information Required/Insufficient I...,600
3,Advanced Imaging Technology (AIT),398
4,Airline,11230


In [13]:
len(df_summary_cat)

49

## 3c. sum incidents overall

In [14]:
# filter to include only rows where category and subcategory are blank
df_totals['Count'].sum()

909409

## 4. compare category totals to subcategory totals

In [15]:
# aggregate subcategories into categories, so they can be compared with category aggregate
df_summary_subcat_aggr = df_summary_subcat.groupby(['Category'])['Count'].sum().reset_index()
df_summary_subcat_aggr.head()

,Category,Count
0,Additional Information Required/Insufficient Inf,504
1,Additional Information Required/Insufficient I...,50
2,Additional Information Required/Insufficient I...,600
3,Advanced Imaging Technology (AIT),398
4,Airline,11230


In [16]:
df_summary_cat['Count'].sum()

909409

In [17]:
df_summary_subcat['Count'].sum()

909409

In [18]:
#test whether the two summaries are identical
df_summary_subcat_aggr.equals(df_summary_cat)

True

## 5. export Results for further review

In [19]:
df_summary_subcat.to_csv("../qa-checks/tsa-summary-subcat.csv")
df_summary_cat.to_csv("../qa-checks/tsa_summary-cat.csv")